In [52]:
import sqlite3
import json
from os import getenv
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
import pytesseract
from PIL import Image
import cv2
import numpy as np
from re import split
import csv

In [53]:
con = sqlite3.connect("custom.db")
cur = con.cursor()

In [54]:
def func(line):
    line = line.strip('"')
    curr_chap, latest_chap = "", ""

    if line[0] == "C": # it starts with "Chapter"
        line = line.split("Chapter")[1:]
        first = line[0].split("Vol")

        if len(first) == 1:
            curr_chap = "Chapter" + line[0][:-1]
            latest_chap = "Chapter " + line[1].strip()
        elif len(first) == 2:
            curr_chap = "Chapter" + first[0][:-1]
            latest_chap = "Vol" + first[1] + "Chapter " + line[-1].strip()

    else: # it starts with "Vol"
        line = line.split("Vol")[1:]

        if len(line) == 1: # the latest chapter starts with "Chapter"
            line = line[0].rsplit(",Chapter", 1)
            curr_chap = "Vol" + line[0]
            latest_chap = "Chapter " + line[-1].strip()
        elif len(line) == 2: # the latest chapter starts with "Vol"
            curr_chap = "Vol" + line[0][:-1]
            latest_chap = "Vol" + line[1].strip()

        # print(f"{curr_chap} --- {latest_chap}")

    return curr_chap, latest_chap

one = "Vol.5 Chapter 43: Cheers,Vol.15 Chapter 165: Takehi Gouda's Ability\n"
two = "Chapter 3,Chapter 11\n"
three = "Chapter 14,Vol.4 Chapter 18\n"
four = "Vol.4 Chapter 14,Chapter 18: Something else is going on\n"

print(func(four))

('Vol.4 Chapter 14', 'Chapter 18: Something else is going on')


In [55]:
titles = []
urls = []
current = []
latest = []
full = []

with open("tempdb.csv", "r", encoding="utf-8") as file:
    csvwriter = csv.reader(file)
    for i, lines in enumerate(file):
        full.append(lines)
        if i == 0:
            continue

        lines = split("https", lines)
        titles.append(lines[0][:-1].strip('"'))

        lines = lines[1].split(",", maxsplit=1)
        urls.append(f"https{lines[0]}")
        
        curr_chap, latest_chap = func(lines[1])
        current.append(curr_chap)
        latest.append(latest_chap)

In [56]:
driver = webdriver.Chrome("..\..\web_scraping\chromedriver-win64\chromedriver.exe")
driver.get("https://asurascans.com")

In [57]:
title = titles[0]
search_form = driver.find_element_by_xpath("//form[contains(@itemtype, 'http://schema.org/SearchAction')]")
search_bar = search_form.find_element_by_xpath(".//input")
search_button = search_form.find_element_by_xpath(".//button")
search_bar.send_keys(title)
search_button.click()

In [71]:
results_name = driver.find_elements_by_xpath(".//div[contains(@class, 'tt')]")[:5]
results_img = driver.find_elements_by_xpath(".//div[contains(@class, 'limit')]/img")
results_img = [r.get_attribute("src") for r in results_img]

https://img.asuracomics.com/unsafe/fit-in/330x450/filters:format(webp)/https://asuratoon.com/wp-content/uploads/2024/05/MargravesBastardSonwasTheEmperorCover02CUnetauto_scaleLevel3width-2000.jpg
https://img.asuracomics.com/unsafe/fit-in/330x450/filters:format(webp)/https://asuratoon.com/wp-content/uploads/2021/07/8thclass.jpg
https://img.asuracomics.com/unsafe/fit-in/330x450/filters:format(webp)/https://asuratoon.com/wp-content/uploads/2021/10/cover.png


In [75]:
all_names = {}

for title in titles:
    search_form = driver.find_element_by_xpath("//form[contains(@itemtype, 'http://schema.org/SearchAction')]")
    search_bar = search_form.find_element_by_xpath(".//input")
    search_button = search_form.find_element_by_xpath(".//button")
    search_bar.send_keys(title)
    search_button.click()

    names = driver.find_elements_by_xpath(".//div[contains(@class, 'tt')]")[:5]
    names = [name.text for name in names]
    all_names[title] = names

for title, results in all_names.items():
    if len(results) > 0:
        print(f"{title}: {results}")

Magic Emperor: ['Margrave’s Bastard Son was The Emperor', 'Return of the 8th Class Magician', 'Return of The Unrivaled Spear Knight']
Helmut: The Forsaken Child: ['Helmut: The Forsaken Child']
Doctor’s Rebirth: ['Doctor’s Rebirth']
Star-Embracing Swordmaster: ['Star-Embracing Swordmaster']
Survival Story of a Sword King in a Fantasy World: ['The Indomitable Martial King']
Son's Retribution: ['Son’s Retribution']
Return Of The Shattered Constellation: ['Return Of The Shattered Constellation']
Insanely Talented Player: ['Insanely-Talented Player']
I am the Strongest Boss: ['Mr Devourer, Please Act Like a Final Boss']
The Hero Returns: ['The Hero Returns', 'The Greatest Sword Hero Returns After 69420 Years', 'The Live', 'Superhuman Battlefield', 'King of the Mound']
My School Life Pretending To Be a Worthless Person: ['My School Life Pretending To Be a Worthless Person', 'Relife Player', 'Dr. Player']
The Knight King Who Returned With a God: ['The Knight King Who Returned with a God', 'I 

In [76]:
driver.close()